In [4]:
# pip install torch transformers datasets scikit-learn matplotlib sentencepiece evaluate

In [5]:
# Import required libraries
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import evaluate

# Load T5 tokenizer and model
def load_t5_model_and_tokenizer(model_name="t5-small"):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_t5_model_and_tokenizer()

# Load SQuAD dataset
dataset = load_dataset("squad", split="train[:5000]")
dataset = dataset.train_test_split(test_size=0.2)

train_subset = dataset["train"].select(range(len(dataset["train"]) // 2))
test_subset = dataset["test"].select(range(len(dataset["test"]) // 2))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
print(dataset)
print(dataset["train"][0])  # İlk örnek

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})
{'id': '56cf318aaab44d1400b88dff', 'title': 'Spectre_(2015_film)', 'context': 'Following Garreth Mallory\'s promotion to M, on a mission in Mexico City unofficially ordered by a posthumous message from the previous M, 007 James Bond kills three men plotting a terrorist bombing during the Day of the Dead and gives chase to Marco Sciarra, an assassin who survived the attack. In the ensuing struggle, Bond steals his ring, which is emblazoned with a stylised octopus, and then kills Sciarra by kicking him out of a helicopter. Upon returning to London, Bond is indefinitely suspended from field duty by M, who is in the midst of a power struggle with C, the head of the privately-backed Joint Intelligence Service, consisting of the recently me

In [7]:
# Preprocess function
def preprocess_squad(examples):
    inputs = ["question: " + q + " context: " + c for q, c in zip(examples["question"], examples["context"])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Cevapların ilk elemanını al
    with tokenizer.as_target_tokenizer():
        labels = tokenizer([ans["text"][0] if ans["text"] else "" for ans in examples["answers"]],
                           max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_subset.map(preprocess_squad, batched=True, remove_columns=dataset["train"].column_names)
tokenized_validation = test_subset.map(preprocess_squad, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [8]:

training_args = TrainingArguments(
    output_dir="./t5_squad_finetuned",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
)


trainer.train()

output_dir = "./models/t5_squad_finetuned2"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model ve tokenizer {output_dir} dizinine başarıyla kaydedildi.")

{'eval_loss': 0.01273129228502512, 'eval_runtime': 18.5316, 'eval_samples_per_second': 26.981, 'eval_steps_per_second': 3.4, 'epoch': 3.0}
{'train_runtime': 963.4793, 'train_samples_per_second': 6.227, 'train_steps_per_second': 0.778, 'train_loss': 0.7061229975422223, 'epoch': 3.0}


Model ve tokenizer ./models/t5_squad_finetuned2 dizinine başarıyla kaydedildi.

In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model and tokenizer
model_path = "./t5_squad_finetuned"
loaded_model = T5ForConditionalGeneration.from_pretrained(model_path)
loaded_tokenizer = T5Tokenizer.from_pretrained(model_path)

In [15]:
# Replace the prediction code in cell 5 with this:
def predict_and_evaluate(test_dataset, model, tokenizer):
    predictions = []
    references = []

    for example in test_dataset:
        question = example['question']
        context = example['context']

        # Format input as expected by T5
        input_text = f"question: {question} context: {context}"
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

        # Generate answer
        outputs = model.generate(**inputs, max_length=64, num_beams=4)
        predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

        predictions.append(predicted_answer)
        references.append(example['answers']['text'][0])

    return predictions, references

# Use the model and tokenizer directly instead of pipeline
predictions, references = predict_and_evaluate(test_subset, loaded_model, loaded_tokenizer)

In [18]:
# pip install rouge_score

In [19]:
import nltk
import evaluate

metric = evaluate.load("squad")
metric_bleu = evaluate.load("bleu")
metric_rouge = evaluate.load("rouge")

formatted_predictions = [{"id": str(i), "prediction_text": p} for i, p in enumerate(predictions)]
formatted_references = [{"id": str(i), "answers": {"text": [r], "answer_start": [0]}} for i, r in enumerate(references)]

results = metric.compute(predictions=formatted_predictions, references=formatted_references)
results_bleu = metric_bleu.compute(predictions=predictions, references=references)
results_rouge = metric_rouge.compute(predictions=predictions, references=references)

print("Evaluation Metrics:")
print(f"Exact Match (EM): {results['exact_match']}")
print(f"F1 Score: {results['f1']}")
print(f"BLEU Score: {results_bleu['bleu']}")
print(f"ROUGE Score: {results_rouge}")

Evaluation Metrics:
Exact Match (EM): 74.6
F1 Score: 84.25407956422669
BLEU Score: 0.5108434245021316
ROUGE Score: {'rouge1': 0.8384149562897176, 'rouge2': 0.5257683189074207, 'rougeL': 0.8396313096430749, 'rougeLsum': 0.8396428977640518}


In [20]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


example_question = "What is the capital of France?"
example_context = "France is a country in Europe. Its capital is Paris, which is known for its culture and history."


input_text = f"question: {example_question} context: {example_context}"


inputs = loaded_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

outputs = loaded_model.generate(**inputs, max_length=64, num_beams=4)
predicted_answer = loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\nExample Prediction:")
print(f"Question: {example_question}")
print(f"Context: {example_context}")
print(f"Predicted Answer: {predicted_answer}")

# Test alt kümesi için tahminler (örnek)
print("\nSample Test Predictions:")
for i in range(10, 15):
    test_example = test_subset[i]
    question = test_example["question"]
    context = test_example["context"]
    true_answer = test_example["answers"]["text"][0]

    # Girdi metni oluşturulup tahmin yapılır
    input_text = f"question: {question} context: {context}"
    inputs = loaded_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = loaded_model.generate(**inputs, max_length=64, num_beams=4)
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Sonuçlar yazdırılır
    print(f"Question: {question}")
    print(f"Context: {context[:100]}...")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("-")



Example Prediction:
Question: What is the capital of France?
Context: France is a country in Europe. Its capital is Paris, which is known for its culture and history.
Predicted Answer: Paris

Sample Test Predictions:
Question: How many deaths were reported only in the Sichuan province?
Context: According to Chinese state officials, the quake caused 69,180 known deaths including 68,636 in Sichu...
True Answer: 68,636
Predicted Answer: 68,636
-
Question: Who said Chopin was unlike his romantic contemporaries Liszt and Henri Herz?
Context: Jones comments that "Chopin's unique position as a composer, despite the fact that virtually everyth...
True Answer: Arthur Hutchings
Predicted Answer: Arthur Hutchings
-
Question: The busiest bus station in the world in the world is called what?
Context: New York City's public bus fleet is the largest in North America, and the Port Authority Bus Termina...
True Answer: Port Authority Bus Terminal
Predicted Answer: Port Authority Bus Terminal
-
Questio